In [ ]:
%pwd

In [ ]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from loguru import logger

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalchatbot"
index = pc.Index(index_name)

In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
def search_db(user_query: str) -> list:
    sim_docs =[]
    result = vector_store.similarity_search_with_score(
    user_query, k=3
    )
    for doc in result:
        sim_docs.append(doc[0].page_content)
    
    return sim_docs    

In [ ]:
search = search_db(user_query="What is candidiasis?")

In [ ]:
search

In [ ]:
# ---------------------------- LLM --------------------------------------
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.5)

In [ ]:
def medicalbot_ai(user_query: str, doc_list: list) -> str:
    template = """
    You are a medical consultant AI chatbot. Your role is to provide accurate and reliable answers to user questions based on the provided documents. Use the information from the `doc_list` to address the `user_query` thoroughly and correctly. Ensure that your response is:

    - **Accurate:** Base your answers solely on the information in the provided documents.
    - **Conversational:** Maintain a friendly and approachable tone.
    - **Mature and Consultancy-Oriented:** Present information in a professional and trustworthy manner.

    **Inputs:**
    1. `user_query`: {user_query} The question posed by the user.
    2. `doc_list`: {doc_list} A list of documents containing relevant information related to the user's question.

    **Instructions:**
    - Analyze the `user_query` and identify the key information needed to answer it.
    - Review the `doc_list` to find relevant information that addresses the query.
    - Construct a response that is clear, concise, and directly answers the user's question using the information from the documents.
    - Avoid introducing information not present in the `doc_list`.
    - If the `user_query` have nothing similar to what is in the `doc_list`, return document not found or something in an apologetic way, tell the user to ask for something related to the context.
    - If the  `user_query` is an empty string, respond with "Please provide a valid query.".
    - Maintain a tone that is both professional and empathetic, suitable for a consultancy setting.
    
    Return the answer as the only output. 
    Always make sure that you're returning the answer without any explanation. 
    The output should be the answer alone.
    Always return this: "Please provide a valid query." for empty query.
    """
    question_prompt = PromptTemplate(input_variables=["user_query", "doc_list"], template=template)
    initiator_router = question_prompt | llm | StrOutputParser()
    output = initiator_router.invoke({"user_query":user_query, "doc_list":doc_list})
    return output

In [ ]:
answer_1 = medicalbot_ai(user_query="What is Candidiasis?", doc_list=search)

In [ ]:
def medical_chatbot(user_query: str)-> str:
    logger.info("Searching for similar Docs in the Vec DB")
    doc_list = search_db(user_query=user_query)
    logger.info("Calling the medicalbot ai to get answer")
    answer = medicalbot_ai(user_query=user_query, doc_list=doc_list)
    logger.info("Returning the Final Answer")
    return answer

In [ ]:
question_1 = "What is Candidiasis?"
question_2 = "what is Carpal tunnel syndrome?"
question_3 = "what is the treatment of Carpal tunnel syndrome?"
question_4 = "What is the treatment of Candidiasis?"

In [ ]:
answer_1 = medical_chatbot(user_query=question_1)
print(answer_1)

In [ ]:
answer_2 = medical_chatbot(user_query=question_2)
print(answer_2)

In [ ]:
answer_3 = medical_chatbot(user_query=question_3)
print(answer_3)

In [ ]:
answer_4 = medical_chatbot(user_query=question_4)
print(answer_4)

In [ ]:
test_5 = medical_chatbot("What is Deep Learning?")
print(test_5)

In [ ]:
test_5 = medical_chatbot("")
print(test_5)